In [2]:
import re
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier


In [10]:
def load_one_flle(filename):
    x=[]
    with open(filename) as f:
        line=f.readline()
        line=line.strip('\n')
    return line

#加载ADFA-LD中的正常样本数据
def load_adfa_training_files(rootdir):
    x=[]
    y=[]
    list = os.listdir(rootdir)# 目录下的文件存为列表
    for i in range(0, len(list)):
        path = os.path.join(rootdir, list[i])#文件名与目录拼接
        if os.path.isfile(path):
            x.append(load_one_flle(path))
            y.append(0)
    return x,y

#定义遍历目录下文件的函数
def dirlist(path, allfile):
    filelist = os.listdir(path)

    for filename in filelist:
        filepath = os.path.join(path, filename)
        if os.path.isdir(filepath):
            dirlist(filepath, allfile)
        else:
            allfile.append(filepath)
    return allfile

#从攻击数据集中筛选和webshell相关的数据
def load_adfa_webshell_files(rootdir):
    x=[]
    y=[]
    allfile=dirlist(rootdir,[])
    for file in allfile:
        #此处小心，前部分分隔符为/，web_shell_x后为\
        if re.match(r"../../Data/ADFA-LD/Attack_Data_Master/Web_Shell_\d+\\UAD-W*",file):
            # print(file)
            x.append(load_one_flle(file))
            y.append(1)
    return x,y


In [21]:

# x1为所有文件中的值,y1为0
x1,y1=load_adfa_training_files("../../Data/ADFA-LD/Training_Data_Master/")
# x2为所有文件中的值,y2为1
x2,y2=load_adfa_webshell_files("../../Data/ADFA-LD/Attack_Data_Master/")
# print(x1[0])
# print("----")
# print(x2[0])
# print("====")
# print(y1[0])
# print("----")
# print(y2[0])
x=x1+x2#拼接
y=y1+y2
# print(x)
vectorizer = CountVectorizer(min_df=1)#将文本文档集合转换为令牌计数矩阵。是一个文本特征提取方法
x=vectorizer.fit_transform(x)
x=x.toarray()
# print(vectorizer.get_feature_names_out())
# #print(y)
clf = KNeighborsClassifier(n_neighbors=3)
scores=cross_val_score(clf, x, y, n_jobs=-1, cv=10)
print(scores)
print(np.mean(scores))

[0.95833333 0.94736842 0.97894737 0.96842105 0.98947368 0.84210526
 0.97894737 0.98947368 0.97894737 0.97894737]
0.9610964912280704
